# Anexo 6: Naive Bayes

En este notebook se realizará la creación del modelo de Naive Bayes, siguiendo los pasos definidos en la documentación para la creación de modelos de clasificación

In [1]:
import pandas as pd
import numpy as np
import feather
import matplotlib as mpl
mpl.use('nbAgg')
import matplotlib.pyplot as plt
from sklearn import preprocessing
import sklearn.model_selection as ms
from sklearn import linear_model
import sklearn.metrics as sklm
from sklearn.preprocessing import label_binarize
import numpy as np
import numpy.random as nr
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as ss
import math
from sklearn.naive_bayes import BernoulliNB
%matplotlib inline

In [2]:
pingfInfoFilePath = './data/dv_personas_190411_abt1'
dv_personas_190411_model = pd.read_feather(pingfInfoFilePath,columns=None, use_threads=True)
dv_personas_190411_model.head()

,sexo,tipo_acceso,tipo_erasmus,edad_fin_primera_titulacion,tto_pcto_dias_trabajados,dni,Titulacion_is_GRADO EN EDUCACIÓN PRIMARIA,Titulacion_is_ING. TÉC. OBRAS PÚB. (ESPECIALIDAD EN CONST. CIV.),Titulacion_is_INGENIERO QUÍMICO,Titulacion_is_INGENIERO TÉCNICO FORESTAL (ESPECIALIDAD EN EXPLOTACIONES FORESTALES),...,Titulacion_is_MÁSTER UNIVERSITARIO EN INVESTIGACIÓN EN HUMANIDADES,Titulacion_is_MÁSTER EN GESTIÓN DEL CONOCIMIENTO BIOMÉDICO E INVESTIGACIÓN CLÍNICA.,Titulacion_is_MÁSTER EN GESTIÓN Y DIRECCIÓN HOSTELERA,Titulacion_is_MÁSTER UNIVERSITARIO EN FORMACIÓN DEL PROFESORADO Y TIC,Titulacion_is_PCEO: DERECHO / ADMINISTRACIÓN Y DIRECCIÓN DE EMPRESAS,Titulacion_is_PCEO: ADMINISTRACIÓN Y DIRECCIÓN DE EMPRESAS / DERECHO,Titulacion_is_PCEO: ADMINISTRACIÓN Y DIRECCIÓN DE EMPRESAS /CIENCIAS DEL TRABAJO,Titulacion_is_MÁSTER UNIVERSITARIO GESTIÓN CONOCIMIENTO BIOMÉDICO E INVESTIGACIÓN CLÍNICA,Titulacion_is_MÁSTER UNIVERSITARIO EN INVESTIGACIÓN EN CIENCIAS SOCIOSANITARIAS,Titulacion_is_MÁSTER UNIVERSITARIO EN PSICOPEDAGOGÍA
0,M,Selectividad,Ninguno,21,23.0,0013670b45b4207d01539bc1d057daa03bab0e2de19916...,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M,Selectividad,Ninguno,26,28.0,2ec47253d8c59fa32f000519d2f9b83c26c7f11dcc1ed8...,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,M,Acceso a Segundo Ciclo,Ninguno,24,21.0,2d2f9c8a7e3c780fef578a2be45f60d537480aeaf01a88...,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,H,Selectividad,Ninguno,21,37.0,df8a544bdeb3b60dfe3c5ab2abce943b2e8769ca4693ce...,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M,Título Universitario,Ninguno,32,78.0,0b7cafeb91821d6c39ccc2f3058edd9c77f351f0047f1a...,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


A continuación, se procede a realizar el proceso de agrupación de datos o data binnning sobre las variables que indican la edad de finalización de la primera titulación y sobre la variable objetivo

In [3]:
dv_personas_190411_model['edad_fin_primera_titulacion'].max()

58

### Agrupación de los datos de la variable edad_fin_primera_titulacion

#### Se procede a agrupar la edad de finalización en rangos de 4 años, comenzando en 21 y terminando en 61

In [4]:
edad_fin_bucket_array = np.linspace(21,61,11)
edad_fin_bucket_array

array([21., 25., 29., 33., 37., 41., 45., 49., 53., 57., 61.])

In [5]:
dv_personas_190411_model['edad_fin_primera_titulacion'] = pd.cut(dv_personas_190411_model['edad_fin_primera_titulacion'],edad_fin_bucket_array,include_lowest=True)
dv_personas_190411_model.head()

,sexo,tipo_acceso,tipo_erasmus,edad_fin_primera_titulacion,tto_pcto_dias_trabajados,dni,Titulacion_is_GRADO EN EDUCACIÓN PRIMARIA,Titulacion_is_ING. TÉC. OBRAS PÚB. (ESPECIALIDAD EN CONST. CIV.),Titulacion_is_INGENIERO QUÍMICO,Titulacion_is_INGENIERO TÉCNICO FORESTAL (ESPECIALIDAD EN EXPLOTACIONES FORESTALES),...,Titulacion_is_MÁSTER UNIVERSITARIO EN INVESTIGACIÓN EN HUMANIDADES,Titulacion_is_MÁSTER EN GESTIÓN DEL CONOCIMIENTO BIOMÉDICO E INVESTIGACIÓN CLÍNICA.,Titulacion_is_MÁSTER EN GESTIÓN Y DIRECCIÓN HOSTELERA,Titulacion_is_MÁSTER UNIVERSITARIO EN FORMACIÓN DEL PROFESORADO Y TIC,Titulacion_is_PCEO: DERECHO / ADMINISTRACIÓN Y DIRECCIÓN DE EMPRESAS,Titulacion_is_PCEO: ADMINISTRACIÓN Y DIRECCIÓN DE EMPRESAS / DERECHO,Titulacion_is_PCEO: ADMINISTRACIÓN Y DIRECCIÓN DE EMPRESAS /CIENCIAS DEL TRABAJO,Titulacion_is_MÁSTER UNIVERSITARIO GESTIÓN CONOCIMIENTO BIOMÉDICO E INVESTIGACIÓN CLÍNICA,Titulacion_is_MÁSTER UNIVERSITARIO EN INVESTIGACIÓN EN CIENCIAS SOCIOSANITARIAS,Titulacion_is_MÁSTER UNIVERSITARIO EN PSICOPEDAGOGÍA
0,M,Selectividad,Ninguno,"(20.999, 25.0]",23.0,0013670b45b4207d01539bc1d057daa03bab0e2de19916...,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M,Selectividad,Ninguno,"(25.0, 29.0]",28.0,2ec47253d8c59fa32f000519d2f9b83c26c7f11dcc1ed8...,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,M,Acceso a Segundo Ciclo,Ninguno,"(20.999, 25.0]",21.0,2d2f9c8a7e3c780fef578a2be45f60d537480aeaf01a88...,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,H,Selectividad,Ninguno,"(20.999, 25.0]",37.0,df8a544bdeb3b60dfe3c5ab2abce943b2e8769ca4693ce...,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M,Título Universitario,Ninguno,"(29.0, 33.0]",78.0,0b7cafeb91821d6c39ccc2f3058edd9c77f351f0047f1a...,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Creación de variables ficticias

#### Se procede a crear las variables ficticias de las diferentes variables categóricas del dataset

In [6]:
categoricalCols = ['sexo','tipo_acceso','tipo_erasmus','edad_fin_primera_titulacion']

In [7]:
dv_personas_190411_model_one_hot = pd.get_dummies(data=dv_personas_190411_model,columns=categoricalCols)

In [8]:
dv_personas_190411_model_one_hot.head()

,tto_pcto_dias_trabajados,dni,Titulacion_is_GRADO EN EDUCACIÓN PRIMARIA,Titulacion_is_ING. TÉC. OBRAS PÚB. (ESPECIALIDAD EN CONST. CIV.),Titulacion_is_INGENIERO QUÍMICO,Titulacion_is_INGENIERO TÉCNICO FORESTAL (ESPECIALIDAD EN EXPLOTACIONES FORESTALES),Titulacion_is_GRADO EN MEDICINA,Titulacion_is_INGENIERO TÉCNICO INDUSTRIAL (ESPECIALIDAD EN ELECTRICIDAD),Titulacion_is_DIPLOMADO EN ENFERMERÍA,Titulacion_is_DIPLOMADO EN TERAPIA OCUPACIONAL,...,"edad_fin_primera_titulacion_(20.999, 25.0]","edad_fin_primera_titulacion_(25.0, 29.0]","edad_fin_primera_titulacion_(29.0, 33.0]","edad_fin_primera_titulacion_(33.0, 37.0]","edad_fin_primera_titulacion_(37.0, 41.0]","edad_fin_primera_titulacion_(41.0, 45.0]","edad_fin_primera_titulacion_(45.0, 49.0]","edad_fin_primera_titulacion_(49.0, 53.0]","edad_fin_primera_titulacion_(53.0, 57.0]","edad_fin_primera_titulacion_(57.0, 61.0]"
0,23.0,0013670b45b4207d01539bc1d057daa03bab0e2de19916...,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,28.0,2ec47253d8c59fa32f000519d2f9b83c26c7f11dcc1ed8...,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,21.0,2d2f9c8a7e3c780fef578a2be45f60d537480aeaf01a88...,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,37.0,df8a544bdeb3b60dfe3c5ab2abce943b2e8769ca4693ce...,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,78.0,0b7cafeb91821d6c39ccc2f3058edd9c77f351f0047f1a...,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [9]:
dv_personas_190411_model_one_hot.drop(['dni'],axis=1,inplace = True)

In [10]:
dv_personas_190411_model_one_hot.head()

,tto_pcto_dias_trabajados,Titulacion_is_GRADO EN EDUCACIÓN PRIMARIA,Titulacion_is_ING. TÉC. OBRAS PÚB. (ESPECIALIDAD EN CONST. CIV.),Titulacion_is_INGENIERO QUÍMICO,Titulacion_is_INGENIERO TÉCNICO FORESTAL (ESPECIALIDAD EN EXPLOTACIONES FORESTALES),Titulacion_is_GRADO EN MEDICINA,Titulacion_is_INGENIERO TÉCNICO INDUSTRIAL (ESPECIALIDAD EN ELECTRICIDAD),Titulacion_is_DIPLOMADO EN ENFERMERÍA,Titulacion_is_DIPLOMADO EN TERAPIA OCUPACIONAL,Titulacion_is_INGENIERO TÉCNICO EN TOPOGRAFÍA,...,"edad_fin_primera_titulacion_(20.999, 25.0]","edad_fin_primera_titulacion_(25.0, 29.0]","edad_fin_primera_titulacion_(29.0, 33.0]","edad_fin_primera_titulacion_(33.0, 37.0]","edad_fin_primera_titulacion_(37.0, 41.0]","edad_fin_primera_titulacion_(41.0, 45.0]","edad_fin_primera_titulacion_(45.0, 49.0]","edad_fin_primera_titulacion_(49.0, 53.0]","edad_fin_primera_titulacion_(53.0, 57.0]","edad_fin_primera_titulacion_(57.0, 61.0]"
0,23.0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,28.0,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,21.0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,37.0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,78.0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


### Agrupación de los datos de la variable objetivo en bloques de 33'333%

In [11]:
tto_pcto_dias_trabajados_bucket_array = np.linspace(0,100,4)
tto_pcto_dias_trabajados_bucket_array

array([  0.        ,  33.33333333,  66.66666667, 100.        ])

In [12]:
dv_personas_190411_model_one_hot_33 = dv_personas_190411_model_one_hot.copy()
dv_personas_190411_model_one_hot_33['tto_pcto_dias_trabajados'] = pd.cut(dv_personas_190411_model_one_hot_33['tto_pcto_dias_trabajados'],tto_pcto_dias_trabajados_bucket_array,include_lowest=True)
dv_personas_190411_model_one_hot_33.head()

,tto_pcto_dias_trabajados,Titulacion_is_GRADO EN EDUCACIÓN PRIMARIA,Titulacion_is_ING. TÉC. OBRAS PÚB. (ESPECIALIDAD EN CONST. CIV.),Titulacion_is_INGENIERO QUÍMICO,Titulacion_is_INGENIERO TÉCNICO FORESTAL (ESPECIALIDAD EN EXPLOTACIONES FORESTALES),Titulacion_is_GRADO EN MEDICINA,Titulacion_is_INGENIERO TÉCNICO INDUSTRIAL (ESPECIALIDAD EN ELECTRICIDAD),Titulacion_is_DIPLOMADO EN ENFERMERÍA,Titulacion_is_DIPLOMADO EN TERAPIA OCUPACIONAL,Titulacion_is_INGENIERO TÉCNICO EN TOPOGRAFÍA,...,"edad_fin_primera_titulacion_(20.999, 25.0]","edad_fin_primera_titulacion_(25.0, 29.0]","edad_fin_primera_titulacion_(29.0, 33.0]","edad_fin_primera_titulacion_(33.0, 37.0]","edad_fin_primera_titulacion_(37.0, 41.0]","edad_fin_primera_titulacion_(41.0, 45.0]","edad_fin_primera_titulacion_(45.0, 49.0]","edad_fin_primera_titulacion_(49.0, 53.0]","edad_fin_primera_titulacion_(53.0, 57.0]","edad_fin_primera_titulacion_(57.0, 61.0]"
0,"(-0.001, 33.333]",1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,"(-0.001, 33.333]",0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,"(-0.001, 33.333]",0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,"(33.333, 66.667]",0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,"(66.667, 100.0]",0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [13]:
dv_personas_190411_model_one_hot_33['tto_pcto_dias_trabajados'].astype('category')

0        (-0.001, 33.333]
1        (-0.001, 33.333]
2        (-0.001, 33.333]
3        (33.333, 66.667]
4         (66.667, 100.0]
               ...       
14315    (-0.001, 33.333]
14316    (-0.001, 33.333]
14317    (-0.001, 33.333]
14318    (-0.001, 33.333]
14319    (-0.001, 33.333]
Name: tto_pcto_dias_trabajados, Length: 14320, dtype: category
Categories (3, interval[float64]): [(-0.001, 33.333] < (33.333, 66.667] < (66.667, 100.0]]

In [14]:
dv_personas_190411_model_one_hot_33['tto_pcto_dias_trabajados'] = dv_personas_190411_model_one_hot_33.tto_pcto_dias_trabajados.cat.codes
dv_personas_190411_model_one_hot_33.head()

,tto_pcto_dias_trabajados,Titulacion_is_GRADO EN EDUCACIÓN PRIMARIA,Titulacion_is_ING. TÉC. OBRAS PÚB. (ESPECIALIDAD EN CONST. CIV.),Titulacion_is_INGENIERO QUÍMICO,Titulacion_is_INGENIERO TÉCNICO FORESTAL (ESPECIALIDAD EN EXPLOTACIONES FORESTALES),Titulacion_is_GRADO EN MEDICINA,Titulacion_is_INGENIERO TÉCNICO INDUSTRIAL (ESPECIALIDAD EN ELECTRICIDAD),Titulacion_is_DIPLOMADO EN ENFERMERÍA,Titulacion_is_DIPLOMADO EN TERAPIA OCUPACIONAL,Titulacion_is_INGENIERO TÉCNICO EN TOPOGRAFÍA,...,"edad_fin_primera_titulacion_(20.999, 25.0]","edad_fin_primera_titulacion_(25.0, 29.0]","edad_fin_primera_titulacion_(29.0, 33.0]","edad_fin_primera_titulacion_(33.0, 37.0]","edad_fin_primera_titulacion_(37.0, 41.0]","edad_fin_primera_titulacion_(41.0, 45.0]","edad_fin_primera_titulacion_(45.0, 49.0]","edad_fin_primera_titulacion_(49.0, 53.0]","edad_fin_primera_titulacion_(53.0, 57.0]","edad_fin_primera_titulacion_(57.0, 61.0]"
0,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,2,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [15]:
dv_personas_190411_model_one_hot_33['tto_pcto_dias_trabajados'].value_counts()

0    9286
1    4013
2    1021
Name: tto_pcto_dias_trabajados, dtype: int64

### División de los datos en características y variable objetivo

In [16]:
Features = dv_personas_190411_model_one_hot_33.iloc[:,1:]
Labels = dv_personas_190411_model_one_hot_33['tto_pcto_dias_trabajados']

In [17]:
Features = np.array(Features)
Features[:2,:]

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [18]:
Features.shape

(14320, 213)

In [19]:
Labels = np.array(Labels)
Labels[:20]

array([0, 0, 0, 1, 2, 0, 1, 2, 0, 1, 2, 1, 1, 0, 0, 2, 0, 1, 2, 2],
      dtype=int8)

In [20]:
nr.seed(123)
inside = ms.KFold(n_splits=10, shuffle = True)
nr.seed(321)
outside = ms.KFold(n_splits=10, shuffle = True)

### Búsueda de mejores híper-parámetros para el modelo

In [21]:
nr.seed(3456)
param_grid = {"alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10]}

NB_clf_ovo_33 = BernoulliNB() 

clf_ovo_33 = ms.GridSearchCV(estimator = NB_clf_ovo_33, param_grid = param_grid, 
                      cv = inside, # Use the inside folds
                      scoring = 'roc_auc_ovo_weighted',
                      return_train_score = True)
clf_ovo_33.fit(Features, Labels)
print(clf_ovo_33.best_estimator_.alpha)

1


### Evaluación del modelo a través del método de validación cruzada

In [22]:
nr.seed(498)
cv_estimate = ms.cross_val_score(clf_ovo_33, Features, Labels, 
                                 cv = outside) # Use the outside folds

print('Mean performance metric = %4.3f' % np.mean(cv_estimate))
print('SDT of the metric       = %4.3f' % np.std(cv_estimate))
print('Outcomes by cv fold')
for i, x in enumerate(cv_estimate):
    print('Fold %2d    %4.3f' % (i+1, x))

Mean performance metric = 0.668
SDT of the metric       = 0.013
Outcomes by cv fold
Fold  1    0.663
Fold  2    0.689
Fold  3    0.684
Fold  4    0.664
Fold  5    0.675
Fold  6    0.647
Fold  7    0.651
Fold  8    0.676
Fold  9    0.672
Fold 10    0.659


### División de los datos en datos de prueba y entrenamiento

In [23]:
x_train, x_test, y_train, y_test = ms.train_test_split(Features,Labels,test_size=0.25, random_state=42)

### Entrenamiento del modelo

In [24]:
NB_mod_ovo_33 = BernoulliNB(alpha=clf_ovo_33.best_estimator_.alpha)
NB_mod_ovo_33.fit(x_train, y_train)
scores = NB_mod_ovo_33.predict(x_test)
scores

array([1, 0, 2, ..., 0, 0, 1], dtype=int8)

### Evaluación del modelo

In [25]:
def print_metrics_33(labels, scores):
    conf = sklm.confusion_matrix(labels, scores)
    print('                 Confusion matrix')
    print('                 Score [0,333.333]    Score (33.333,66.667] Score (66.667,100]')
    print('Actual [0,333.333]        %6d' % conf[0,0] + '             %5d' % conf[0,1] + '             %5d' % conf[0,2])
    print('Actual (33.333,66.667]    %6d' % conf[1,0] + '             %5d' % conf[1,1] + '             %5d' % conf[1,2])
    print('Actual (66.667,100]       %6d' % conf[2,0] + '             %5d' % conf[2,1] + '             %5d' % conf[2,2])

    print('')
    print('Accuracy        %0.2f' % sklm.accuracy_score(labels, scores))
    metrics = sklm.precision_recall_fscore_support(labels, scores)
    print(' ')
    print('         Score [0,333.333]    Score (33.333,66.667] Score (66.667,100]')
    print('Num case   %0.2f' % metrics[3][0] + '                  %0.2f' % metrics[3][1] + '                  %0.2f' % metrics[3][2] )
    print('Precision   %0.2f' % metrics[0][0] + '                      %0.2f' % metrics[0][1] + '                   %0.2f' % metrics[0][2] )
    print('Recall      %0.2f' % metrics[1][0] + '                      %0.2f' % metrics[1][1] + '                   %0.2f' % metrics[1][2] )
    print('F1          %0.2f' % metrics[2][0] + '                      %0.2f' % metrics[2][1] + '                   %0.2f' % metrics[2][2] )

In [26]:
print_metrics_33(y_test, scores)  

                 Confusion matrix
                 Score [0,333.333]    Score (33.333,66.667] Score (66.667,100]
Actual [0,333.333]          1945               152               177
Actual (33.333,66.667]       754               127               163
Actual (66.667,100]          150                32                80

Accuracy        0.60
 
         Score [0,333.333]    Score (33.333,66.667] Score (66.667,100]
Num case   2274.00                  1044.00                  262.00
Precision   0.68                      0.41                   0.19
Recall      0.86                      0.12                   0.31
F1          0.76                      0.19                   0.23


### Agrupación de los datos de la variable objetivo en bloques de 25%

In [29]:
dv_personas_190411_model_one_hot[dv_personas_190411_model_one_hot['tto_pcto_dias_trabajados'] == 0]

,tto_pcto_dias_trabajados,Titulacion_is_GRADO EN EDUCACIÓN PRIMARIA,Titulacion_is_ING. TÉC. OBRAS PÚB. (ESPECIALIDAD EN CONST. CIV.),Titulacion_is_INGENIERO QUÍMICO,Titulacion_is_INGENIERO TÉCNICO FORESTAL (ESPECIALIDAD EN EXPLOTACIONES FORESTALES),Titulacion_is_GRADO EN MEDICINA,Titulacion_is_INGENIERO TÉCNICO INDUSTRIAL (ESPECIALIDAD EN ELECTRICIDAD),Titulacion_is_DIPLOMADO EN ENFERMERÍA,Titulacion_is_DIPLOMADO EN TERAPIA OCUPACIONAL,Titulacion_is_INGENIERO TÉCNICO EN TOPOGRAFÍA,...,"edad_fin_primera_titulacion_(20.999, 25.0]","edad_fin_primera_titulacion_(25.0, 29.0]","edad_fin_primera_titulacion_(29.0, 33.0]","edad_fin_primera_titulacion_(33.0, 37.0]","edad_fin_primera_titulacion_(37.0, 41.0]","edad_fin_primera_titulacion_(41.0, 45.0]","edad_fin_primera_titulacion_(45.0, 49.0]","edad_fin_primera_titulacion_(49.0, 53.0]","edad_fin_primera_titulacion_(53.0, 57.0]","edad_fin_primera_titulacion_(57.0, 61.0]"
23,0.0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
45,0.0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
66,0.0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
67,0.0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
81,0.0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14152,0.0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
14230,0.0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
14285,0.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
14298,0.0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [30]:
tto_pcto_dias_trabajados_bucket_array = np.linspace(0,100,5)
tto_pcto_dias_trabajados_bucket_array

array([  0.,  25.,  50.,  75., 100.])

In [31]:
dv_personas_190411_model_one_hot_25 = dv_personas_190411_model_one_hot.copy()
dv_personas_190411_model_one_hot_25['tto_pcto_dias_trabajados'] = pd.cut(dv_personas_190411_model_one_hot_25['tto_pcto_dias_trabajados'],tto_pcto_dias_trabajados_bucket_array,include_lowest=True)
dv_personas_190411_model_one_hot_25.head()

,tto_pcto_dias_trabajados,Titulacion_is_GRADO EN EDUCACIÓN PRIMARIA,Titulacion_is_ING. TÉC. OBRAS PÚB. (ESPECIALIDAD EN CONST. CIV.),Titulacion_is_INGENIERO QUÍMICO,Titulacion_is_INGENIERO TÉCNICO FORESTAL (ESPECIALIDAD EN EXPLOTACIONES FORESTALES),Titulacion_is_GRADO EN MEDICINA,Titulacion_is_INGENIERO TÉCNICO INDUSTRIAL (ESPECIALIDAD EN ELECTRICIDAD),Titulacion_is_DIPLOMADO EN ENFERMERÍA,Titulacion_is_DIPLOMADO EN TERAPIA OCUPACIONAL,Titulacion_is_INGENIERO TÉCNICO EN TOPOGRAFÍA,...,"edad_fin_primera_titulacion_(20.999, 25.0]","edad_fin_primera_titulacion_(25.0, 29.0]","edad_fin_primera_titulacion_(29.0, 33.0]","edad_fin_primera_titulacion_(33.0, 37.0]","edad_fin_primera_titulacion_(37.0, 41.0]","edad_fin_primera_titulacion_(41.0, 45.0]","edad_fin_primera_titulacion_(45.0, 49.0]","edad_fin_primera_titulacion_(49.0, 53.0]","edad_fin_primera_titulacion_(53.0, 57.0]","edad_fin_primera_titulacion_(57.0, 61.0]"
0,"(-0.001, 25.0]",1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,"(25.0, 50.0]",0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,"(-0.001, 25.0]",0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,"(25.0, 50.0]",0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,"(75.0, 100.0]",0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [32]:
dv_personas_190411_model_one_hot_25['tto_pcto_dias_trabajados'].astype('category')

0        (-0.001, 25.0]
1          (25.0, 50.0]
2        (-0.001, 25.0]
3          (25.0, 50.0]
4         (75.0, 100.0]
              ...      
14315    (-0.001, 25.0]
14316    (-0.001, 25.0]
14317    (-0.001, 25.0]
14318    (-0.001, 25.0]
14319    (-0.001, 25.0]
Name: tto_pcto_dias_trabajados, Length: 14320, dtype: category
Categories (4, interval[float64]): [(-0.001, 25.0] < (25.0, 50.0] < (50.0, 75.0] < (75.0, 100.0]]

In [33]:
dv_personas_190411_model_one_hot_25['tto_pcto_dias_trabajados'] = dv_personas_190411_model_one_hot_25.tto_pcto_dias_trabajados.cat.codes
dv_personas_190411_model_one_hot_25.head()

,tto_pcto_dias_trabajados,Titulacion_is_GRADO EN EDUCACIÓN PRIMARIA,Titulacion_is_ING. TÉC. OBRAS PÚB. (ESPECIALIDAD EN CONST. CIV.),Titulacion_is_INGENIERO QUÍMICO,Titulacion_is_INGENIERO TÉCNICO FORESTAL (ESPECIALIDAD EN EXPLOTACIONES FORESTALES),Titulacion_is_GRADO EN MEDICINA,Titulacion_is_INGENIERO TÉCNICO INDUSTRIAL (ESPECIALIDAD EN ELECTRICIDAD),Titulacion_is_DIPLOMADO EN ENFERMERÍA,Titulacion_is_DIPLOMADO EN TERAPIA OCUPACIONAL,Titulacion_is_INGENIERO TÉCNICO EN TOPOGRAFÍA,...,"edad_fin_primera_titulacion_(20.999, 25.0]","edad_fin_primera_titulacion_(25.0, 29.0]","edad_fin_primera_titulacion_(29.0, 33.0]","edad_fin_primera_titulacion_(33.0, 37.0]","edad_fin_primera_titulacion_(37.0, 41.0]","edad_fin_primera_titulacion_(41.0, 45.0]","edad_fin_primera_titulacion_(45.0, 49.0]","edad_fin_primera_titulacion_(49.0, 53.0]","edad_fin_primera_titulacion_(53.0, 57.0]","edad_fin_primera_titulacion_(57.0, 61.0]"
0,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,1,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [34]:
dv_personas_190411_model_one_hot_25['tto_pcto_dias_trabajados'].value_counts()

0    7497
1    4319
2    1966
3     538
Name: tto_pcto_dias_trabajados, dtype: int64

### División de los datos en características y variable objetivo

In [35]:
Features = dv_personas_190411_model_one_hot_25.iloc[:,1:]
Labels = dv_personas_190411_model_one_hot_25['tto_pcto_dias_trabajados']

In [36]:
Features = np.array(Features)
Features[:2,:]

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [37]:
Features.shape

(14320, 213)

In [38]:
Labels = np.array(Labels)
Labels[:20]

array([0, 1, 0, 1, 3, 0, 1, 3, 0, 2, 2, 1, 1, 0, 0, 3, 0, 1, 2, 3],
      dtype=int8)

In [39]:
nr.seed(123)
inside = ms.KFold(n_splits=10, shuffle = True)
nr.seed(321)
outside = ms.KFold(n_splits=10, shuffle = True)

### Búsueda de mejores híper-parámetros para el modelo

In [40]:
nr.seed(3456)
param_grid = {"alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10]}

NB_clf_ovo_25 = BernoulliNB() 

clf_ovo_25 = ms.GridSearchCV(estimator = NB_clf_ovo_25, param_grid = param_grid, 
                      cv = inside, 
                      scoring = 'roc_auc_ovo_weighted',
                      return_train_score = True)

clf_ovo_25.fit(Features, Labels)
print(clf_ovo_25.best_estimator_.alpha)

0.1


### Evaluación del modelo a través del método de validación cruzada

In [41]:
nr.seed(498)
cv_estimate = ms.cross_val_score(clf_ovo_25, Features, Labels, 
                                 cv = outside) 

print('Mean performance metric = %4.3f' % np.mean(cv_estimate))
print('SDT of the metric       = %4.3f' % np.std(cv_estimate))
print('Outcomes by cv fold')
for i, x in enumerate(cv_estimate):
    print('Fold %2d    %4.3f' % (i+1, x))

Mean performance metric = 0.658
SDT of the metric       = 0.011
Outcomes by cv fold
Fold  1    0.654
Fold  2    0.666
Fold  3    0.679
Fold  4    0.647
Fold  5    0.661
Fold  6    0.647
Fold  7    0.646
Fold  8    0.654
Fold  9    0.672
Fold 10    0.649


### División de los datos en datos de prueba y entrenamiento

In [42]:
x_train, x_test, y_train, y_test = ms.train_test_split(Features,Labels,test_size=0.25, random_state=42)

### Entrenamiento del modelo

In [43]:
NB_mod_ovo_25 = BernoulliNB(alpha=clf_ovo_25.best_estimator_.alpha)
NB_mod_ovo_25.fit(x_train, y_train)
scores = NB_mod_ovo_25.predict(x_test)
scores

array([2, 0, 3, ..., 0, 0, 2], dtype=int8)

### Evaluación del modelo


In [57]:
def print_metrics_25(labels, scores):
    conf = sklm.confusion_matrix(labels, scores)
    print('                 Confusion matrix')
    print('                 Score [0,25]    Score (25,50]   Score (50,75]   Score (75,100]')
    print('Actual [0,25]     %6d' % conf[0,0] + '             %5d' % conf[0,1] + '        %5d' % conf[0,2] + '          %5d' % conf[0,3])
    print('Actual (25,50]    %6d' % conf[1,0] + '             %5d' % conf[1,1] + '        %5d' % conf[1,2] + '          %5d' % conf[1,3])
    print('Actual (50,75]    %6d' % conf[2,0] + '             %5d' % conf[2,1] + '        %5d' % conf[2,2] + '          %5d' % conf[2,3])
    print('Actual (75,100]   %6d' % conf[3,0] + '             %5d' % conf[3,1] + '        %5d' % conf[3,2] + '          %5d' % conf[3,3])

    print('')
    print('Accuracy        %0.2f' % sklm.accuracy_score(labels, scores))
    metrics = sklm.precision_recall_fscore_support(labels, scores)
    print(' ')
    print('          Score [0,25]  Score (25,50]  Score (50,75] Score (75,100]')
    print('Num case   %0.2f' % metrics[3][0] + '         %0.2f' % metrics[3][1] + '        %0.2f' % metrics[3][2] + '       %0.2f' % metrics[3][3] )
    print('Precision   %0.2f' % metrics[0][0] + '            %0.2f' % metrics[0][1] + '           %0.2f' % metrics[0][2] + '         %0.2f' % metrics[0][3] )
    print('Recall      %0.2f' % metrics[1][0] + '            %0.2f' % metrics[1][1] + '           %0.2f' % metrics[1][2] + '         %0.2f' % metrics[1][3] )
    print('F1          %0.2f' % metrics[2][0] + '            %0.2f' % metrics[2][1] + '           %0.2f' % metrics[2][2] + '         %0.2f' % metrics[2][3] )

In [58]:
print_metrics_25(y_test, scores)  

                 Confusion matrix
                 Score [0,25]    Score (25,50]   Score (50,75]   Score (75,100]
Actual [0,25]       1502               110          135             82
Actual (25,50]       796               123          117             82
Actual (50,75]       335                28           71             68
Actual (75,100]       63                 6           38             24

Accuracy        0.48
 
          Score [0,25]  Score (25,50]  Score (50,75] Score (75,100]
Num case   1829.00         1118.00        502.00       131.00
Precision   0.56            0.46           0.20         0.09
Recall      0.82            0.11           0.14         0.18
F1          0.66            0.18           0.16         0.12


### Agrupación de los datos de la variable objetivo en bloques de 20%

In [60]:
tto_pcto_dias_trabajados_bucket_array = np.linspace(0,100,6)
tto_pcto_dias_trabajados_bucket_array

array([  0.,  20.,  40.,  60.,  80., 100.])

In [61]:
dv_personas_190411_model_one_hot_20 = dv_personas_190411_model_one_hot.copy()
dv_personas_190411_model_one_hot_20['tto_pcto_dias_trabajados'] = pd.cut(dv_personas_190411_model_one_hot_20['tto_pcto_dias_trabajados'],tto_pcto_dias_trabajados_bucket_array,include_lowest=True)
dv_personas_190411_model_one_hot_20.head()

,tto_pcto_dias_trabajados,Titulacion_is_GRADO EN EDUCACIÓN PRIMARIA,Titulacion_is_ING. TÉC. OBRAS PÚB. (ESPECIALIDAD EN CONST. CIV.),Titulacion_is_INGENIERO QUÍMICO,Titulacion_is_INGENIERO TÉCNICO FORESTAL (ESPECIALIDAD EN EXPLOTACIONES FORESTALES),Titulacion_is_GRADO EN MEDICINA,Titulacion_is_INGENIERO TÉCNICO INDUSTRIAL (ESPECIALIDAD EN ELECTRICIDAD),Titulacion_is_DIPLOMADO EN ENFERMERÍA,Titulacion_is_DIPLOMADO EN TERAPIA OCUPACIONAL,Titulacion_is_INGENIERO TÉCNICO EN TOPOGRAFÍA,...,"edad_fin_primera_titulacion_(20.999, 25.0]","edad_fin_primera_titulacion_(25.0, 29.0]","edad_fin_primera_titulacion_(29.0, 33.0]","edad_fin_primera_titulacion_(33.0, 37.0]","edad_fin_primera_titulacion_(37.0, 41.0]","edad_fin_primera_titulacion_(41.0, 45.0]","edad_fin_primera_titulacion_(45.0, 49.0]","edad_fin_primera_titulacion_(49.0, 53.0]","edad_fin_primera_titulacion_(53.0, 57.0]","edad_fin_primera_titulacion_(57.0, 61.0]"
0,"(20.0, 40.0]",1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,"(20.0, 40.0]",0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,"(20.0, 40.0]",0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,"(20.0, 40.0]",0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,"(60.0, 80.0]",0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [62]:
dv_personas_190411_model_one_hot_20['tto_pcto_dias_trabajados'].astype('category')

0          (20.0, 40.0]
1          (20.0, 40.0]
2          (20.0, 40.0]
3          (20.0, 40.0]
4          (60.0, 80.0]
              ...      
14315    (-0.001, 20.0]
14316    (-0.001, 20.0]
14317    (-0.001, 20.0]
14318      (20.0, 40.0]
14319    (-0.001, 20.0]
Name: tto_pcto_dias_trabajados, Length: 14320, dtype: category
Categories (5, interval[float64]): [(-0.001, 20.0] < (20.0, 40.0] < (40.0, 60.0] < (60.0, 80.0] < (80.0, 100.0]]

In [63]:
dv_personas_190411_model_one_hot_20['tto_pcto_dias_trabajados'] = dv_personas_190411_model_one_hot_20.tto_pcto_dias_trabajados.cat.codes
dv_personas_190411_model_one_hot_20.head()

,tto_pcto_dias_trabajados,Titulacion_is_GRADO EN EDUCACIÓN PRIMARIA,Titulacion_is_ING. TÉC. OBRAS PÚB. (ESPECIALIDAD EN CONST. CIV.),Titulacion_is_INGENIERO QUÍMICO,Titulacion_is_INGENIERO TÉCNICO FORESTAL (ESPECIALIDAD EN EXPLOTACIONES FORESTALES),Titulacion_is_GRADO EN MEDICINA,Titulacion_is_INGENIERO TÉCNICO INDUSTRIAL (ESPECIALIDAD EN ELECTRICIDAD),Titulacion_is_DIPLOMADO EN ENFERMERÍA,Titulacion_is_DIPLOMADO EN TERAPIA OCUPACIONAL,Titulacion_is_INGENIERO TÉCNICO EN TOPOGRAFÍA,...,"edad_fin_primera_titulacion_(20.999, 25.0]","edad_fin_primera_titulacion_(25.0, 29.0]","edad_fin_primera_titulacion_(29.0, 33.0]","edad_fin_primera_titulacion_(33.0, 37.0]","edad_fin_primera_titulacion_(37.0, 41.0]","edad_fin_primera_titulacion_(41.0, 45.0]","edad_fin_primera_titulacion_(45.0, 49.0]","edad_fin_primera_titulacion_(49.0, 53.0]","edad_fin_primera_titulacion_(53.0, 57.0]","edad_fin_primera_titulacion_(57.0, 61.0]"
0,1,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,1,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [64]:
dv_personas_190411_model_one_hot_20['tto_pcto_dias_trabajados'].value_counts()

0    6184
1    4261
2    2374
3    1127
4     374
Name: tto_pcto_dias_trabajados, dtype: int64

### División de los datos en características y variable objetivo

In [65]:
Features = dv_personas_190411_model_one_hot_20.iloc[:,1:]
Labels = dv_personas_190411_model_one_hot_20['tto_pcto_dias_trabajados']

In [66]:
Features = np.array(Features)
Features[:2,:]

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [67]:
Features.shape

(14320, 213)

In [68]:
Labels = np.array(Labels)
Labels[:20]

array([1, 1, 1, 1, 3, 0, 1, 3, 0, 3, 3, 2, 2, 0, 0, 3, 0, 2, 3, 3],
      dtype=int8)

In [57]:
nr.seed(123)
inside = ms.KFold(n_splits=10, shuffle = True)
nr.seed(321)
outside = ms.KFold(n_splits=10, shuffle = True)

### Búsueda de mejores híper-parámetros para el modelo

In [69]:
nr.seed(3456)
## Define the dictionary for the grid search and the model object to search on
param_grid = {"alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10]}
## Define the NB regression model
NB_clf_ovo_20 = BernoulliNB() 

## Perform the grid search over the parameters
clf_ovo_20 = ms.GridSearchCV(estimator = NB_clf_ovo_20, param_grid = param_grid, 
                      cv = inside, # Use the inside folds
                      scoring = 'roc_auc_ovo_weighted',
                      return_train_score = True)
clf_ovo_20.fit(Features, Labels)
print(clf_ovo_20.best_estimator_.alpha)

1


### Evaluación del modelo a través del método de validación cruzada

In [70]:
nr.seed(498)
cv_estimate = ms.cross_val_score(clf_ovo_20, Features, Labels, 
                                 cv = outside) # Use the outside folds

print('Mean performance metric = %4.3f' % np.mean(cv_estimate))
print('SDT of the metric       = %4.3f' % np.std(cv_estimate))
print('Outcomes by cv fold')
for i, x in enumerate(cv_estimate):
    print('Fold %2d    %4.3f' % (i+1, x))

Mean performance metric = 0.645
SDT of the metric       = 0.012
Outcomes by cv fold
Fold  1    0.634
Fold  2    0.670
Fold  3    0.662
Fold  4    0.643
Fold  5    0.650
Fold  6    0.633
Fold  7    0.639
Fold  8    0.646
Fold  9    0.647
Fold 10    0.631


### División de los datos en datos de prueba y entrenamiento

In [71]:
x_train, x_test, y_train, y_test = ms.train_test_split(Features,Labels,test_size=0.25, random_state=42)

### Entrenamiento del modelo

In [72]:
NB_mod_ovo_20 = BernoulliNB(alpha=clf_ovo_20.best_estimator_.alpha)
NB_mod_ovo_20.fit(x_train, y_train)
scores = NB_mod_ovo_20.predict(x_test)
scores

array([3, 0, 3, ..., 0, 0, 2], dtype=int8)

### Evaluación del modelo

In [94]:
def print_metrics_20(labels, scores):
    conf = sklm.confusion_matrix(labels, scores)
    print('                 Confusion matrix')
    print('                 Score [0,20]    Score (20,40] Score (40,60] Score (60,80] Score (80,100]')
    print('Actual [0,20]     %6d' % conf[0,0] + '           %5d' % conf[1,1] + '            %5d' % conf[1,2] + '         %5d' % conf[1,3] + '        %5d' % conf[1,4])
    print('Actual (40,60]   %6d' % conf[2,0] + '            %5d' % conf[2,1] + '            %5d' % conf[2,2] + '         %5d' % conf[2,3] + '        %5d' % conf[2,4])
    print('Actual (60,80]   %6d' % conf[3,0] + '            %5d' % conf[3,1] + '            %5d' % conf[3,2] + '         %5d' % conf[3,3] + '        %5d' % conf[3,4])
    print('Actual (80,100]  %6d' % conf[4,0] + '            %5d' % conf[4,1] + '            %5d' % conf[4,2] + '         %5d' % conf[4,3] + '        %5d' % conf[4,4])

    print('')
    print('Accuracy        %0.2f' % sklm.accuracy_score(labels, scores))
    metrics = sklm.precision_recall_fscore_support(labels, scores)
    print(' ')
    print('          Score [0,20]  Score (20,40]  Score (40,60]  Score (60,80]  Score (80,100]')
    print('Num case   %0.2f' % metrics[3][0] + '         %0.2f' % metrics[3][1] + '         %0.2f' % metrics[3][2] + '          %0.2f' % metrics[3][3] + '       %0.2f' % metrics[3][4])
    print('Precision   %0.2f' % metrics[0][0] + '            %0.2f' % metrics[0][1] + '            %0.2f' % metrics[0][2] + '            %0.2f' % metrics[0][3] + '         %0.2f' % metrics[0][4])
    print('Recall      %0.2f' % metrics[1][0] + '            %0.2f' % metrics[1][1] + '            %0.2f' % metrics[1][2] + '            %0.2f' % metrics[1][3] + '         %0.2f' % metrics[1][4])
    print('F1          %0.2f' % metrics[2][0] + '            %0.2f' % metrics[2][1] + '            %0.2f' % metrics[2][2] + '            %0.2f' % metrics[2][3] + '         %0.2f' % metrics[2][4])

In [95]:
print_metrics_20(y_test, scores)  

                 Confusion matrix
                 Score [0,20]    Score (20,40] Score (40,60] Score (60,80] Score (80,100]
Actual [0,20]       1168             162               69            76           43
Actual (40,60]      379               80               49            74           37
Actual (60,80]      170               16               26            50           35
Actual (80,100]      42                1                9            20           13

Accuracy        0.40
 
          Score [0,20]  Score (20,40]  Score (40,60]  Score (60,80]  Score (80,100]
Num case   1522.00         1057.00         619.00          297.00       85.00
Precision   0.47            0.38            0.23            0.16         0.08
Recall      0.77            0.15            0.08            0.17         0.15
F1          0.59            0.22            0.12            0.16         0.10
